In [2]:
!python cria_grafo.py

Grafo densamente conectado gerado e salvo em 'grafo.txt'.


In [3]:
!g++ Wall -O3 main.cpp -o main

'g++' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
